In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# Any results you write to the current directory are saved as output.

In [2]:
import torch.utils.data as D
from PIL import Image
from torchvision import transforms,models

class ImagesDS(D.Dataset):
    def __init__(self, df, site=1, channels=[1,2,3,4,5,6]):
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.img_dir = '../input'
        self.len = df.shape[0]
        
    @staticmethod
    def _load_img_as_tensor(file_name):
        with Image.open(file_name) as img:
            img = transforms.Resize((224,224))(img)
            img = transforms.CenterCrop(224)(img)
            return transforms.ToTensor()(img)

    def _get_img_path(self, index, channel):
        experiment, well, plate = self.records[index].experiment, self.records[index].well, self.records[index].plate
        return '/'.join([self.img_dir,'train',experiment,f'Plate{plate}',f'{well}_s{self.site}_w{channel}.png'])
        
    def __getitem__(self, index):
        paths = [self._get_img_path(index, ch) for ch in self.channels]
        img = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths])
        return img, int(self.records[index].sirna)

    def __len__(self):
        return self.len
        

In [3]:
df = pd.read_csv('/kaggle/input/train.csv')
train1 = ImagesDS(df)
train2 = ImagesDS(df,site=2)
trainset = D.ConcatDataset([train1,train2])


In [4]:
from torch.utils.data.sampler import SubsetRandomSampler
import torch

dataset_size = len(trainset)
indices = list(range(dataset_size))
split = int(np.floor(0.2 * dataset_size))
np.random.seed(42)
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, 
                                           sampler=train_sampler,num_workers=4)
val_loader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                                sampler=valid_sampler,num_workers=4)

In [5]:
from torch import optim,nn
model = models.densenet201(pretrained=True)
model.features.conv0 = nn.Conv2d(6,64,kernel_size=(7,7),stride=(2,2),padding=(3,3),bias=False)
model.classifier = nn.Sequential(nn.BatchNorm1d(1920),
                                nn.Linear(1920,1108),
                                nn.LogSoftmax(dim=1))
device = torch.device("cuda:0")
model.to(device)

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /tmp/.cache/torch/checkpoints/densenet201-c1103571.pth
100%|██████████| 81131730/81131730 [00:01<00:00, 59804133.23it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [6]:
from sklearn.metrics import accuracy_score
optimizer = optim.Adam(model.parameters(),lr=0.003)
criterion = nn.CrossEntropyLoss()
epochs = 13
for i in range(epochs):
    tr_loss = 0
    v_loss = 0
    for im,y in train_loader:
        im,y = im.to(device),y.to(device)
        optimizer.zero_grad()
        output = model(im)
        p_y = torch.exp(output)
        loss = criterion(output,y)
        loss.backward()
        optimizer.step()
        tr_loss+=loss.item()
    with torch.no_grad():
        model.eval()
        for im,y in val_loader:
            im,y=im.to(device),y.to(device)
            output = model(im)
            p_y = torch.exp(output)
            loss = criterion(output,y)
            v_loss+=loss.item()
        model.train()
    print(f'Epoch:{i+1}  TrainLoss;{tr_loss/len(train_loader)}  ValLoss:{v_loss/len(val_loader)}')
    #print(f'TrainAccuracy:{acc} ValAcc:{vacc}')
    print('---------------------------------------------------------------------------------------------')
    
torch.save(model,'./model.pth')

        
    

Epoch:1  TrainLoss;6.964105798276335  ValLoss:7.762087278345221
---------------------------------------------------------------------------------------------
Epoch:2  TrainLoss;6.599314707393583  ValLoss:8.329003552117547
---------------------------------------------------------------------------------------------
Epoch:3  TrainLoss;6.3068830883385445  ValLoss:6.587489033162724
---------------------------------------------------------------------------------------------
Epoch:4  TrainLoss;6.012329015648247  ValLoss:6.320427501227715
---------------------------------------------------------------------------------------------
Epoch:5  TrainLoss;5.713810472864441  ValLoss:6.0739257048725825
---------------------------------------------------------------------------------------------
Epoch:6  TrainLoss;5.458364441256403  ValLoss:5.991001921096568
---------------------------------------------------------------------------------------------
Epoch:7  TrainLoss;5.211106364854977  ValLoss:6.11